<a href="https://colab.research.google.com/github/emmanuel-mejia/Matematicas-para-Ciencia-de-Datos/blob/main/semana6_Algebra_Textrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementación de TextRank para la obtención de resúmenes

En este Notebook se implementará TextRank para obtener un resumen con las oraciones clave de todo un texto.

# Dependencias

In [1]:
%%capture
!pip install wikipedia git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [2]:
# PUEDE ser necesario utilizar una versión anterior de pillow
!pip install Pillow==9.0.0

  Using cached Pillow-9.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.1.0
    Uninstalling Pillow-9.1.0:
      Successfully uninstalled Pillow-9.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
import re

import pandas as pd
import numpy as np
import scipy.linalg as splinalg

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import wikipedia

from txtai.pipeline import Translation

In [2]:
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Radicalizador
stemmer = PorterStemmer()

# Palabras de paro
cached_stopwords = stopwords.words('english')
cached_stopwords[:10]

# Traductor
translate = Translation()

# Datos

Los datos que ocuparemos serán el texto de páginas de Wikipedia. Descargaremos el texto ocupando el módulo [```wikipedia```](https://pypi.org/project/wikipedia/) que es un "wrapper" del API de Wikipedia. A este texto lo dividiremos en oraciones, procesaremos cada oración, radicalizaremos cada palabra, y aplicaremos TextRank para obtener las oraciones más importantes de todo el documento.

## Lectura de los datos

Descargamos un artículos de Wikipedia.

In [4]:
wiki = wikipedia.page('Expropiación del petróleo en México')
book = wiki.content
print(book)

The Mexican oil expropriation (Spanish: expropiación petrolera) was the nationalization of all petroleum reserves, facilities, and foreign oil companies in Mexico on March 18, 1938.  In accordance with Article 27 of the Constitution of 1917, President Lázaro Cárdenas declared that all mineral and oil reserves found within Mexico belong to "the nation", i.e., the federal government. The Mexican government established a state-owned petroleum company, Petróleos Mexicanos, or PEMEX.  For a short period, this measure caused an international boycott of Mexican products in the following years, especially by the United States, the United Kingdom, and the Netherlands, but with the outbreak of World War II and the alliance between Mexico and the Allies, the disputes with private companies over compensation were resolved. The anniversary, March 18, is now a Mexican civic holiday.


== Background ==

On August 16, 1935, the Petroleum Workers Union of Mexico (Sindicato de Trabajadores Petroleros de

## Procesamiento

Dividimos el texto en oraciones.

In [ ]:
sentences = [x for x in sent_tokenize(book)]
print(f"# oraciones: {len(sentences)}")
for sentence in sentences[:3]:
    print(sentence)
    print()
    print("...Fin de la oración...")
    print()


convertimos a minúsculas, eliminamos stopwords, eliminamos signos de puntuación y radicalizamos.

In [ ]:
sent_low = [[stemmer.stem(re.sub('[^a-z]', "", word.lower())) for word in word_tokenize(sentence) if word not in cached_stopwords and len(word) > 2] for sentence in sentences]
sent_low[1]

# TextRank

Construimos la matriz de adyacencias/similitud A entre las oraciones, tomando el número de palabras que están en ambas como la similitud entre las dos oraciones.

In [ ]:
A = np.zeros((len(sent_low), len(sent_low)))

for i in range(len(sentences)):
    if i % 100 == 0:
        print(i, end=", ")
        if i % 1000 == 0:
            print()
    for j in range(i+1, len(sentences)):
        # La simillitud entre oraciones va a ser el número de palabras que tienen en común
        A[i][j] = A[j][i] = len([x for x in sentences[i] if x in sentences[j]])

Así es como se ve un fragmento de la matriz A.

In [ ]:
A[:5, :5]

Normalizamos las columnas de A

In [ ]:
# Comparamos las oraciones unas con otras, pero no consigo mismas
suma = np.sum(A, axis=0)
A_norm = np.divide(A, suma, where=suma!=0)
A_norm[:5, :5]

Se crea el vector de TextRank con unos y se itera hasta que converja. Es decir, hasta que obtengamos $\Pi$ tal que $$\Pi = A~\Pi$$ 

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
tol = 1e-7

PI_ = np.ones(A_norm.shape[1])
    
i = 0
while True:
    pi_ = A_norm.dot(PI_)
    print(i, abs(PI_- pi_).sum()) 
    if np.allclose(PI_, pi_, tol):
        break
    i += 1
    PI_ = pi_

Alternativamente, podemos obtener los eigenvectores izquierdos de nuestra matriz A_norm. Los valores de PageRank corresponden al vector de probabilidades del estado estacionario de la matriz A que a su vez es el eigenvector izquierdo con eigenvalor asociado 1.

$$\Pi = \Pi A^T$$

In [ ]:
_, vecs = splinalg.eig(A_norm.T, left=True, right=False)

In [ ]:
pi_ = vecs[:, 0]
pi_

Obtenemos los índices de los k valores más grandes en $\Pi$ y los usamos para obtener las oraciones más relevantes.

In [ ]:
k = 10
pi_.argsort()[-k:][::-1]

In [ ]:
summary = [sentences[idx] for idx in pi_.argsort()[-k:][::-1]]

In [ ]:
summary

Por último, sólo queda ver qué considero TextRank como las oraciones más importantes.

In [ ]:
for bullet in summary:
    print('___________')
    print(bullet)

Podemos traducir la salida.

In [ ]:
# Aprox 34 seg las primeras 10 oraciones
for bullet in summary:
    print()
    print(translate(bullet, "es"))

# Función para crear resúmenes

Podemos condensar todo lo anterior en una función que reciba texto y nos regrese las oraciones más relevantes de acuerdo a TextRank.

In [ ]:
def summary(text, k, to_spanish = True, tol = 1e-5, d = .15, eig = False):
    print("Paso 1. Obteniendo oraciones")
    sentences = [x for x in sent_tokenize(text)]

    print(f"# oraciones: {len(sentences)}")
    
    print("Paso 2. Procesando texto")
    sent_low = [[stemmer.stem(re.sub('[^a-z]', "", word.lower())) for word in word_tokenize(sentence) if word not in cached_stopwords and len(word) > 2] for sentence in sentences]
    
    print("Paso 3. Creando matriz de similitud")
    A = np.zeros((len(sent_low), len(sent_low)))
    
    for i in range(len(sentences)):
        for j in range(i+1, len(sentences)):
            # La simillitud entre oraciones va a ser el número de palabras que tienen en común
            A[i][j] = A[j][i] = len([x for x in sentences[i] if x in sentences[j]])

    print("Paso 4. Normalizando matriz de similitud")   
    suma = np.sum(A, axis=0)
    A_norm = np.divide(A, suma, where=suma!=0)
    
    print("Paso 5. Ejecutando TextRank")
    if eig:
        vals, vecs = splinalg.eig(A_norm.T, left=True, right=False)
        pi_ = vecs[:, 0]
    else:
        PI_ = np.ones(A_norm.shape[1])
        
        while True:
            pi_ = A_norm.dot(PI_)
            if np.allclose(PI_, pi_, tol):
                break
            PI_ = pi_

    print("\tPaso 5. Terminado")

    if not to_spanish:
        return [sentences[idx] for idx in pi_.argsort()[-k:][::-1]]

    print("Paso 6. Traduciendo")
    return [translate(sentences[idx], "es") for idx in pi_.argsort()[-k:][::-1]]

def print_bullet_points(bullet_points):
    for point in bullet_points:
        print(f"- {point}\n")


In [ ]:
wiki = wikipedia.page('Automatic summarization')
text = wiki.content
bullet_points = summary(text, 5, False, eig = False)

In [ ]:
print_bullet_points(bullet_points)

In [ ]:
!wget https://www.gutenberg.org/files/84/84-0.txt -O book.txt

In [ ]:
with open("book.txt") as f:
    book_raw = f.read()
print(book_raw[0:1000])

In [ ]:
start = book_raw.rfind("Chapter 5\n")
end = book_raw.rfind('Chapter 6\n')

In [ ]:
chapter_n = book_raw[start + len("Chapter 5\n"): end]

In [ ]:
bullet_points = summary(chapter_n, 5, False, eig = True)

In [ ]:
print_bullet_points(bullet_points)

# Ejercicios

## Matriz de similitud entre oraciones

Para la similitud entre las oraciones se uso el número de palabras que aparecen en ambas. **Reemplazar por similitud coseno** y comparar los resultados.

Un muy buen primer acercamiento podría ser usando Latent Semantic Analysis y calcular la similitud coseno entre todos los documentos.

Si tienen una DataFrame con las columnas ```[id_documento_1, id_documento_2, similitud]```, usar la función [```pandas.DataFrame.pivot```](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) puede ayudar a crear la matriz de similitud, dicha función toma como argumentos "index", "columns" y "values".




## Idioma

Este ejemplo esta hecho para texto en inglés por las stopwords que se usan y el radicalizador (PorterStemmer). Hacer los cambios necesarios para que reciba textos en español.

Esto es, cambiar las stopwords (nltk tiene stopwords en español) y el radicalizador (Pista: ```nltk.stemmer``` tiene más radicalizadores y uno de ellos tienen un algoritmo para el español)

## Oraciones vs. Palabras

En este Notebook utilizamos las oraciones para obtener el resumen, de haber utilizado las palabras, de TextRank obtendríamos las palabras clave del texto. 

Implementar TextRank con palabras. Para la matriz de similitud (o adyacencias), se pueden ligar las palabras que son consecutivas o definir una ventana de k palabras consecutivas en cada oración (parecido a skip-gram) y ligar todas estas palabras. En este caso, la matriz A tendría la dimensión del vocabulario (lista de palabras únicas) y tendría un 1 si las palabras están ligadas.

Una alternativa más sería ocupar un embedding de palabras (e.g. word2vec) y calcular la similitud coseno entre los vectores de cada palabra para llenas a A.

Después de eso, todo sería lo mismo.

## Resumen sobre un tema

Aquí usamos sólo un documento para aplicarle TextRank. Podemos tener un corpus de documentos del mismo tema (e.g. noticias sobre el AIFA, etc) y aplicarlo para obtener los puntos importantes de todo el corpus.

A la implementación actual no se le tiene que cambiar nada, sólo concatenar en una sola cadena de texto todo el corpus.

Ejercicio: Construir un corpus con 4 artículos sobre un tema de interés, concatenarlos y pasarlo como parámetro a la función ```summary```.

## Mejorar la función ```summary```

Podemos dividir el código de la función para que funcionen como módulos y permita cierta libertad a la hora de ejecutarse. Por ejemplo, podríamos tener varias funciones que calculen la matriz A de diferentes maneras y que dentro de ```summary``` se ejecute una de tantas de acuerdo a un parámetro de la función.

Ejercicio: Crear funciones para cada paso de ```summary```

# Sobre la obtención de los valores de PageRank

https://nlp.stanford.edu/IR-book/html/htmledition/the-pagerank-computation-1.html

https://nlp.stanford.edu/IR-book/html/htmledition/markov-chains-1.html